In [35]:
from lxml import etree
import base64
import sqlite3
import pandas as pd
import cairosvg
from PyPDF2 import PdfMerger
from datetime import datetime, timedelta
from io import BytesIO
import os
from modules.data_utils import fetch_agent_contact_info

# Define namespaces
NSMAP = {
    None: "http://www.w3.org/2000/svg",
    "xlink": "http://www.w3.org/1999/xlink",
    "inkscape": "http://www.inkscape.org/namespaces/inkscape",
    "sodipodi": "http://sodipodi.sourceforge.net/DTD/sodipodi-0.dtd"
}

def load_svg(input_svg_path):
    """
    Load and parse an SVG file.

    Args:
        input_svg_path (str): Path to the input SVG file.

    Returns:
        tuple: A tuple containing the parsed tree and root element, or (None, None) if an error occurs.
    """
    try:
        tree = etree.parse(input_svg_path)
        root = tree.getroot()
        return tree, root
    except FileNotFoundError:
        print(f"Error: SVG file not found at '{input_svg_path}'")
    except etree.XMLSyntaxError as e:
        print(f"Error parsing SVG file: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    return None, None

def replace_text(root, element_id, old_text, new_text, namespaces):
    text_element = root.find(f".//*[@id='{element_id}']", namespaces=namespaces)
    if text_element is not None:
        tspan_x = text_element.get("x")
        tspan_y = text_element.get("y")
        updated_text = text_element.text.replace(old_text, str(new_text))
        text_element.text = updated_text
        if tspan_x is not None:
            text_element.set("x", tspan_x)
        if tspan_y is not None:
            text_element.set("y", tspan_y)
        # Ensure parent <tspan> or <text> doesn’t override
        parent = text_element.getparent()
        if parent.tag == "{http://www.w3.org/2000/svg}tspan":
            parent_x = parent.get("x")
            parent_y = parent.get("y")
            if parent_x is not None:
                parent.set("x", parent_x)
            if parent_y is not None:
                parent.set("y", parent_y)
        #print(f"Stabilized <tspan> at x={tspan_x}, y={tspan_y}")

def embed_svg(root, base_svg_path, embed_svg_path, x=0, y=0, scale=1.0):
    try:
        # Parse the SVG to embed
        embed_tree = etree.parse(embed_svg_path)
        embed_root = embed_tree.getroot()

        # Create a <g> element to group the embedded SVG content
        group = etree.Element("g", nsmap=NSMAP)
        transform = f"translate({x}, {y}) scale({scale})"
        group.set("transform", transform)

        # Move all children from the embed SVG's root to the group
        for child in embed_root:
            group.append(child)

        # Insert the group at the beginning of the root's children
        root.insert(0, group)  # Changed from append to insert at index 0

        print(f"SVG embedded successfully at the start of the root")
        return root.getroottree()

    except etree.XMLSyntaxError as e:
        print(f"Error parsing embedded SVG: {e}")
    except FileNotFoundError as e:
        print(f"Error: Embedded SVG file not found - {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None


In [61]:
# Add this before saving the tree at the end of process_page7
def flatten_svg_structure(root):
    """Flatten the SVG structure to ensure proper layer ordering."""
    # Find all nested SVGs that were embedded
    embedded_svgs = root.findall(".//svg:svg", namespaces=NSMAP)
    
    # For each embedded SVG, extract its elements and add them directly to the parent
    for embedded_svg in embedded_svgs:
        parent = embedded_svg.getparent()
        
        # Create a group to contain all elements from the embedded SVG
        group = etree.Element("{%s}g" % NSMAP['svg'])
        
        # Copy any transform from the embedded SVG to the new group
        if embedded_svg.get("transform"):
            group.set("transform", embedded_svg.get("transform"))
        
        # Move all elements from embedded SVG to the group
        for element in list(embedded_svg):
            group.append(element)
        
        # Replace the embedded SVG with the group in the parent
        parent.replace(embedded_svg, group)

# After embedding all SVGs, ensure proper stacking order
def enforce_layer_order(root):
    """Enforce proper layer ordering of elements."""
    # Find the main container element (often a group or the root)
    container = root
    
    # Sort elements by their intended z-index if specified
    elements = list(container)
    
    # Reorder elements based on priority
    # Remove all elements from container
    for element in elements:
        container.remove(element)
    
    # Add them back in correct order
    for element in elements:
        container.append(element)

def process_page7(IN, media_mensal, total_contrato, economia_contratual, economia_anual, input_svg_path="Proposta PPT/page 7.svg", output_svg_path="Temp_ppt/page 7.svg", db_path="DataBase.db"):
    
    validade = datetime.today() + timedelta(days=5)

    # Load the SVG file
    tree, root = load_svg(input_svg_path)
    if not tree or not root:
        return
    
    # Replace text in the SVG file
    replace_text(root, "tspan520-74-4-0-6", "XXXXXXXXXX", IN, NSMAP) 
    replace_text(root, "tspan520-7", "xx xxx,xx", f"{media_mensal:,.2f}".replace(",", " ").replace(".",","), NSMAP) 
    replace_text(root, "tspan520-7-1", "xx xxx,xx", f"{total_contrato:,.2f}".replace(",", " ").replace(".",","), NSMAP)
    replace_text(root, "tspan5", "XXX XXX", f"{total_contrato:,.0f}".replace(",", " ").replace(".",","), NSMAP)
    replace_text(root, "tspan1", "25%",  f"{economia_contratual:.0%}" , NSMAP)    
    #replace_text(root, "tspan2","14%", f"{economia_efetiva:.0%}" , NSMAP)
    replace_text(root, "tspan12","20/02/2025", f"{validade.day:02d}/{validade.month:02d}/{validade.year}", NSMAP)  
    embed_svg(root, input_svg_path,"images/energy_cost_plot.svg" ,x=125,y=75, scale= 0.18)
    embed_svg(root, input_svg_path,"images/historic_graph.svg" ,x=80,y=170, scale= 0.165)
    
    # Save the modified SVG file
    try:
        #flatten_svg_structure(root)  # Flatten the SVG structure before saving
        #enforce_layer_order(root)  # Ensure proper layer ordering of elements
        tree.write(output_svg_path, pretty_print=True, xml_declaration=True, encoding="utf-8")
        print(f"Modified SVG saved to '{output_svg_path}'")

    except IOError as e:
        print(f"Error saving modified SVG: {e}")
    
    except Exception as e:
        print(f"Unexpected error: {e}")

process_page7("30411253", 4370.00, 157370.00, 0.26, 0.12)

SVG embedded successfully at the start of the root
SVG embedded successfully at the start of the root
Modified SVG saved to 'Temp_ppt/page 7.svg'


C:\Users\Rodri\AppData\Local\Temp\ipykernel_37336\2530711646.py:49: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not tree or not root:


In [62]:
from modules.pdf_builder import generate_pdf, open_pdf

generate_pdf(["Temp_ppt/page 7.svg"], "Temp_ppt/page7.pdf")
open_pdf(r"C:\Users\Rodri\Documents\Apresentacao-Energia-Livre\Temp_ppt\page7.pdf")

Converted Temp_ppt/page 7.svg to PDF stream
Merged PDF saved as Temp_ppt/page7.pdf


In [29]:
from modules.pdf_builder import process_page5

In [30]:
process_page5("30411253", 4370.00, 157370.00, 0.26, 0.12)
generate_pdf(["Temp_ppt/page 5.svg"], "Temp_ppt/page 5.pdf")

c:\Users\Rodri\Documents\Apresentacao-Energia-Livre\modules\pdf_builder.py:224: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not tree or not root:


SVG embedded successfully
Modified SVG saved to 'Temp_ppt/page 5.svg'
Converted Temp_ppt/page 5.svg to PDF stream
Merged PDF saved as Temp_ppt/page 5.pdf


In [1]:
from modules.plot_generator import create_historic_graph

In [9]:

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
vs_cativo = [620, 610, 630, 600, 640, 650, 660, 670, 680, 690, 700, 710]
v_desconto = [12, 14, 13, 15, 14, 13, 12, 14, 13, 15, 14, 13]
vs_fio_acl = [0.75 * cativo - desconto for cativo, desconto in zip(vs_cativo, v_desconto)]

create_historic_graph(months, vs_cativo, v_desconto, vs_fio_acl, figsize=(16,5))